In [1]:
from bls_py import (bls, ec)
from random import randint

We will work in the G1 (Fq) group since public keys are in the G1 group.

In [2]:
G = ec.generator_Fq() # G1
l = ec.default_ec.n   # the G1 group order

In [3]:
def Hp(point: bls.AffinePoint) -> bls.AffinePoint:
    """Hash a point to point"""
    return ec.hash_to_point_Fq(point.serialize())

def Hs(*xs):
    """Hash to field, result will be an integer in 0..l"""
    h = b''
    for x in xs:
        if type(x) == str:
            b = bytes(x, encoding="utf-8")
        elif type(x) == bls.AffinePoint:
            b = x.serialize()
        else:
            b = bytes(x)

        h = ec.hash256(h + b)

    return int.from_bytes(h, "big") % l

In [4]:
# Number of signers in the ring
N = 21

# The message we are signing
m = "important message"

xs = [bls.PrivateKey.from_seed(str(i)).value for i in range(N)]
P = [x * G for x in xs]

# j is the index of the secret signer in the ring
j = 0
x = xs[0]

In [5]:
# Generate the random blinding values
alpha = randint(0, l)
s = [randint(0, l) if i != j else None for i in range(N)]

In [6]:
# Key Image
I = x * Hp(P[j])

# Prepare for ring signing
L = [None] * N
R = [None] * N
c = [None] * N

# Start at index j, then proceed around the ring.
# Each step computes c[i+1]
L[j] = alpha * G
R[j] = alpha * Hp(P[j])
c[(j+1) % N] = Hs(m, L[j], R[j])

for offset in range(1, N):
    i = (j + offset) % N
    L[i] = s[i] * G + c[i] * P[i]
    R[i] = s[i] * Hp(P[i]) + c[i] * I
    c[(i + 1) % N] = Hs(m, L[i], R[i])

# s[j] is chosen so that it completes the ring without exposing x
s[j] = (alpha - c[j] * x) % l

$s_j = \alpha - c_j x  \text{ mod }  l$

rearranging for alpha gives:

(1) $\alpha = s_j + c_j x  \text{ mod }  l$

Now consider the equation for L[j], R[j] used above:

$L_j = \alpha G$

$R_j = \alpha H_p(P_j)$

Substituing $\alpha$ with (1):

$L_j = \alpha G = (s_j + c_j x) G = s_j G + c_j x G = s_j G + c_j P_j$


$R_j = \alpha H_p(P_j) = s_j H_p(P_j) + c_j x H_p(P_j) =  s_j H_p(P_j) + c_j I$

The result is $s_j$ and $c_j$ can be exposed without revealing $x$

In [7]:
sig = (I, c[0], s)

In [8]:
# Sanity check on invariants
assert alpha == (s[j] + c[j] * x) % l
assert L[j] == alpha * G
assert alpha * G == s[j] * G + c[j] * x * G
assert alpha * G == s[j] * G + (c[j] * x % l) * G
assert c[j] * x  * G == c[j] * P[j]
assert alpha * G == s[j] * G + c[j] * P[j]

In [9]:
# Now to verify
def verify(I, c0, s, P, m):
    N = len(P)
    
    Lp = [None] * N
    Rp = [None] * N
    cp = [None] * N
    cp[0] = c0

    for i in range(N):
        Lp[i] = s[i] * G + cp[i] * P[i]
        Rp[i] = s[i] * Hp(P[i]) + cp[i] * I
        cp[(i + 1) % N] = Hs(m, Lp[i], Rp[i])
        
    assert cp[0] == c0
    assert Hs(m, Lp[-1], Rp[-1]) == cp[0]
    print("Everything checks out")

In [10]:
verify(*sig, P, m)

Everything checks out
